In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%env HOME=/content/drive/MyDrive/
#!mkdir -p ~/Research/huggingface
%cd ~/Research/ner/area_templates/

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner/area_templates


In [ ]:
%cd ~/Research/ner/DSTC10_evaluation/

/content/drive/MyDrive/Research/ner/DSTC10_evaluation


In [ ]:
!pip install fastDamerauLevenshtein
!pip install num2words

  Using cached fastDamerauLevenshtein-1.0.7-cp37-cp37m-linux_x86_64.whl
  Using cached num2words-0.5.10-py3-none-any.whl (101 kB)


In [ ]:
import json
f1 = open('/content/drive/MyDrive/Research/ner/DSTC10_evaluation/DSTC10_task2_log_eval_phon_templates.json', 'r')
log = json.load(f1)
f1.close()

f2 = open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/db.json','r')
db = json.load(f2)
f2.close()

f3 = open('/content/drive/MyDrive/Research/ner/DSTC10_track2_task2_log_phon_templates.json','r')
log2 = json.load(f3)
f3.close()

In [ ]:
#%%writefile BabyTrie_DSTC10_v6_area.py
import re
import json
from fastDamerauLevenshtein import damerauLevenshtein as dl
from num2words import num2words
from simple_tokenize import Clean_Text
from simple_tokenize import Word_Tokenize
from extra_methods import simple_num2words

class BabyTrie:#baby version of Trie
    class TrieNode:#Node within a Trie
        def __init__(self,entity=None):
            self.children={}#dictionary of TrieNodes
            self.markers=[]#list of markers/categories
            self.end=False#the Node is leaf, or end of word
            self.entity=None#in case needed
            self.id=None
    
    def __init__(self):
        self.root=self.new_node()

    def new_node(self,word=None):
        #Creates TrieNode object with a given word
        return self.TrieNode(word)

    def restricted_insert(self,lst,cat,number = -1):
        if len(lst) == 0:
            print("ERROR: empty NE in knowledge file")
            return
        if len(lst) == 1 and lst[0]==cat:
            print("only one elem in lst")
            return
        
        if lst[0]=='the':#the is ambiguous
            lst.pop(0)

        ptr=self.root
        for i in range(0,len(lst)):#for every elem but not the last (since it is the category)
            if lst[i] not in ptr.children.keys():#if already a key, skip; else add new
                new=self.new_node(lst[i])
                ptr.children[lst[i]]=new
            ptr=ptr.children[lst[i]]
        #if (len(lst)==2 and lst[-2] in ['good','ask','page']):#we dont want to treat good or ask as an entity (see test.txt)
            #ptr.end=False
        if (len(lst)==3 and lst[-3]=='cable' and lst[-2]=='car'):
            ptr.end=False
        else:
            ptr.end=True#is end of word
            ptr.id=number
        if cat not in ptr.markers:#Only adds new categories to the list of markers 
            ptr.markers.append(cat)
    
    def insert(self,lst,cat,number):
        if len(lst) == 0:
            print("ERROR: empty NE in knowledge file")
            return
        if len(lst) == 1 and lst[0]==cat:
            print("only one elem in lst")
            return
        
        if lst[0]=='the':#the is ambiguous
            lst.pop(0)
        
        
        
        if lst[-1]!=cat: #['a','and','b'] + ['restaurant']
            lst.append(cat)
        ptr=self.root
        for i in range(0,len(lst)-1):#for every elem but not the last (since it is the category)
            if lst[i] not in ptr.children.keys():#if already a key, skip; else add new
                new=self.new_node(lst[i])
                ptr.children[lst[i]]=new
            ptr=ptr.children[lst[i]]
        if (len(lst)==2 and lst[-2] in ['good','ask','page']):#we dont want to treat good or ask as an entity (see test.txt)
            ptr.end=False
        elif (len(lst)==3 and lst[-3]=='cable' and lst[-2]=='car'):
            ptr.end=False
        else:
            ptr.end=True#is end of word
            ptr.id=number
        if cat not in ptr.markers:#Only adds new categories to the list of markers 
            ptr.markers.append(cat)
        if lst[-1] not in ptr.children.keys():#avoid possible conflict with same name
            new=self.new_node(lst[-1])
            ptr.children[lst[-1]]=new#adds the category as one of the leaf node
        ptr=ptr.children[lst[-1]]
        ptr.end=True
        ptr.id=number
        if cat not in ptr.markers:#only adds category if not in list of markers
            ptr.markers.append(cat)
    
    def isinTrie(self,sen):
        stutter_match=False
        dl_mistakes=['train','want','american','lane','fees','fee','marina','wanna','canna','dinna','finna']
        #outfile for tracking log process
        dic={}#the uncleaned version of returned dictionary
        flag=False#=True when a phrase in the sentence does not match the trie anymore
        lst=[]#the list that will ultimately be returned

        new_sen=sen.lower().strip()#cleans up sentence

        lst=Word_Tokenize(Clean_Text(new_sen))
        
        last=len(lst)-1#keep track of last to ensure index does not go over limit
        #the returned list is completely formed at this point, we use it to generate dictionary
        rstart=0#starting position of NE
        rend=0#ending position of NE

        proceed=False
        for i in range(len(lst)):
            ptr=self.root
            if lst[i] in ptr.children:
                proceed=True
                rstart=i
                ptr=ptr.children[lst[i]]
            elif lst[i] not in ptr.children and lst[i] not in dl_mistakes:
                for word in ptr.children:
                    if len(lst[i])<=4:
                        dl_score=dl(lst[i],word,similarity=False,deleteWeight=2,insertWeight=2,replaceWeight=2)
                        bm=0
                    else:
                        dl_score=dl(lst[i],word,similarity=False,deleteWeight=1,insertWeight=2,replaceWeight=2)
                        bm=1
                    if dl_score<=bm:
                        proceed=True
                        rstart=i
                        ptr=ptr.children[word]
                        break
            if proceed:
                stutter_utt=0
                for j in range(i+1,len(lst)):
                    if lst[j] in ptr.children:
                        ptr=ptr.children[lst[j]]
                        rend=j
                        stutter_match=False
                    elif lst[j] not in ptr.children:
                        proceed=False
                        for word in ptr.children:
                            dl_score=dl(lst[j],word,similarity=False,deleteWeight=2,insertWeight=2,replaceWeight=2)
                            bm=1
                            if dl_score<=bm and lst[j].isalnum():
                                ptr=ptr.children[word]
                                rend=j
                                proceed=True
                                stutter_match=False
                                break
                        if (len(lst[j])<=3) and (lst[j].isalnum()) and (stutter_utt<1):#allow one mismatch (except for punc), could be stuttering
                            stutter_utt+=1
                            proceed=True
                            rend=j-1
                            stutter_match=True
                            #here rend not counted since if next is not matched, it will not be effected if the prev word is actually end of word
                        elif stutter_match:
                            rend=j-2
                            break
                        #print('for '+lst[i]+' stopped at '+lst[j])#degbug use
                        if not proceed:
                            rend=j-1
                            break

            if ptr.end:
                for i in range(len(ptr.markers)):
                    if (ptr.markers[i] in dic):#if already a key, add to value
                        if rstart>rend:
                            rend=rstart
                        dic[ptr.markers[i]].append(((rstart,rend),ptr.id))
                    else:#if not, make new entry
                        if rstart>rend:
                            rend=rstart
                        dic[ptr.markers[i]]=[((rstart,rend),ptr.id)]
                rstart=0
                rend=0
            else:
                continue
        #cleaning up dictionary
        rdic={}#clean version of the dictionary that is ultimately returned
        for cat in dic.keys():
            rdic[cat]=[]
            rlst=sorted(dic[cat],key=lambda item:item[0][1]-item[0][0],reverse=True)
            #rdic[cat].append(rlst[0])
            for i in range(0,len(rlst)):
                rflag=True
                for rcat in rdic.keys():#make sure no overlap: ex. 'some hotel diner' > 'some hotel'
                    for elem in rdic[rcat]:
                        if (rlst[i][0][0]<elem[0][0] and rlst[i][0][1]<elem[0][0]):#ex. if we have (3,5) we can take (1,2)
                            continue
                        elif (rlst[i][0][0]>elem[0][1] and rlst[i][0][1]>elem[0][1]):#ex. if we have (3,5) we can take (6,7)
                            continue
                        else:# any equals, [0] or [1] will not work: ex. we have (3,5). cannot have (3,4) or (4,5).
                            if rlst[i][0][1]-rlst[i][0][0]==elem[0][1]-elem[0][0] and rlst[i][0][1]>elem[0][1]:
                                rdic[rcat].remove(elem)
                                continue
                            else:
                                rflag=False
                if rflag:
                    rdic[cat].append(rlst[i])
        del_lst=[]#cheap(costly) way to deal with empty dic entry
        for cat in rdic:
            if len(rdic[cat])==0:
                del_lst.append(cat)
        for elem in del_lst:
            del rdic[elem]
        return (lst,rdic)

    def initialize_area(self,dic):#add db_dic if-/>
        #extract ref knowledge from db_dic TBDDDDDDDDDD
        #start of initialization
        dig=False
        chain_entities={}#keeping track of chain restaurants' ids i.e. {'entity name':('cat','id1|id2|id3|...')}
        for cat in dic.keys():
            for record in dic[cat]:

                if ('area' in record and record['area'] is not None):
                    new_name=record['area'].lower().strip()#cleans up name
                    if new_name == cat:
                        print('exceptional entity---------------------')
                        continue
                    lst=[]#final list that is used to insert into bt
                    dlst=[]
                    alt_dlst=[]
                    no_punc_lst=[]
                    plst=Word_Tokenize(Clean_Text(new_name))
                    #p=re.compile(r"^(\w+)(\'\w+)$")

                    for i in range (len(plst)):
                        if plst[i].isnumeric():
                            dig=True
                            for alt_dig in plst[i]:
                                alt_dlst.append(simple_num2words(int(alt_dig)))

                            converted=num2words(plst[i])
                            if '-' in converted:
                                converted_lst=converted.split('-')
                                for digword in converted_lst:
                                    dlst.append(digword)
                            else:
                                converted_lst=converted.split()
                                for digword in converted_lst:
                                    dlst.append(digword)
                            lst.append(plst[i])
                            continue
                        else:
                            lst.append(plst[i])
                            dlst.append(plst[i])
                            alt_dlst.append(plst[i])
                    self.restricted_insert(lst,'area','-1')
                    #self.restricted_insert(no_punc_lst,cat,elem)#'alpha-milton guest house'

                    if dig:
                        self.restricted_insert(dlst,'area','-1')
                        self.restricted_insert(alt_dlst,'area','-1')

[nltk_data] Downloading package punkt to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
bt = BabyTrie()
bt.initialize_area(db)

ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file
ERROR: empty NE in knowledge file


In [ ]:
bt.isinTrie("Fisherman's Wharf baby")

(['fisherman', "'s", 'wharf', 'baby'], {'area': [((0, 2), '-1')]})

In [ ]:
def gettemplate_area_map(title):
    word_lst,ind_dic = bt.isinTrie(title)
    nlst = Word_Tokenize(Clean_Text(title))
    new_sen = title.strip()#cleans up sentence
    en_dic={}

    n = 1
    for key in ind_dic:
        ind_lst=ind_dic[key]
        for pair in ind_lst:
            temp_str=''
            for index in range(pair[0][0],pair[0][1]+1):
                if not nlst[index][0].isalnum():
                    temp_str=temp_str[:-1]
                if nlst[index]=='-':
                    temp_str+=nlst[index]
                else:
                    temp_str+=nlst[index]+' '
            temp_str=temp_str[:-1]#getting rid of last space
            #temp_str_converted=convert_text2num(temp_str)#name entity to be in dig form
            en_dic['<'+key+'-'+str(n)+'>']=temp_str
            title=title.replace(temp_str,'<'+key+'-'+str(n)+'>')
            n += 1
    return [title,en_dic]

In [ ]:
for dialogue in log2:
    for turn in dialogue:
        area_template, area_map = gettemplate_area_map(turn['template'])
        turn['area_template'] = area_template
        turn['area_map'] = area_map
out = open('DSTC10_task2_log_template_warea.json','w')
json.dump(log2, out, indent = 4)
out.close()